In [1]:
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import os
import sys
import pickle

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Dynamic path settings
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
DATA_DIR = os.path.join(BASE_DIR, "dataset")
TRAIN_TEST_SPLIT_DIR = os.path.join(BASE_DIR, "train_test_split")

In [4]:
# Upload Maps
with open(os.path.join(DATA_DIR, 'user_id_map.pkl'), 'rb') as f:
    user_id_map = pickle.load(f)

with open(os.path.join(DATA_DIR, 'product_id_map.pkl'), 'rb') as f:
    product_id_map = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pavin\\Documents\\dataset\\user_id_map.pkl'

In [5]:
spark = SparkSession.builder \
    .appName("RecommenderSystem") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.cores", "4") \
    .config("spark.python.worker.reuse", "false") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

In [6]:
# Load test data
train_sales_path = os.path.join(TRAIN_TEST_SPLIT_DIR, "train_sales_data.csv")
test_sales_path = os.path.join(TRAIN_TEST_SPLIT_DIR, "test_sales_data.csv")
train_df = pd.read_csv(train_sales_path)
test_df = pd.read_csv(test_sales_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pavin\\Documents\\train_test_split\\train_sales_data.csv'

In [7]:
# Check common user and product IDs
common_user_ids = set(train_df['user_id']).intersection(set(test_df['user_id']))
common_product_ids = set(train_df['product_id']).intersection(set(test_df['product_id']))

print(f"Common user IDs: {len(common_user_ids)}")
print(f"Common product IDs: {len(common_product_ids)}")

if len(common_user_ids) == 0 or len(common_product_ids) == 0:
    raise ValueError("No common user IDs or product IDs between training and test sets.")

NameError: name 'train_df' is not defined

In [8]:
# Ensure there are data rows in test data
if test_df.empty:
    raise ValueError("Test DataFrame is empty.")
print("Test DataFrame:\n", test_df.head(), "\n")

NameError: name 'test_df' is not defined

In [9]:
# Convert interaction_type column to numeric
test_df['interaction_type'] = pd.to_numeric(test_df['interaction_type'], errors='coerce')
test_df.dropna(subset=['interaction_type'], inplace=True)

NameError: name 'test_df' is not defined

In [10]:
# Check if the dataframe is still empty after conversion and dropping NA values
if test_df.empty:
    raise ValueError("Test DataFrame is empty after converting 'interaction_type' to numeric and dropping NA values.")

NameError: name 'test_df' is not defined

In [11]:
# Convert test data to Spark DataFrame
test_data = [Row(user_id=int(row.user_id), product_id=int(row.product_id), interaction_type=float(row['interaction_type'])) for index, row in test_df.iterrows()]
test_df_spark = spark.createDataFrame(test_data)

NameError: name 'test_df' is not defined

In [12]:
print("Test DataFrame:\n", test_df_spark.head(), "\n")

NameError: name 'test_df_spark' is not defined

In [13]:
# Load the trained model using Spark's load method
model_path = os.path.join(BASE_DIR, "als_model")
model = ALSModel.load(model_path)

Py4JJavaError: An error occurred while calling o42.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/C:/Users/pavin/Documents/als_model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.RDD.$anonfun$take$1(RDD.scala:1471)

In [14]:
# Making predictions on test data
predictions = model.transform(test_df_spark)

NameError: name 'model' is not defined

In [15]:
if predictions.head(1) == []:
    raise ValueError("Predictions DataFrame is empty after model.transform(). Ensure your test data has sufficient entries.")

NameError: name 'predictions' is not defined

In [16]:
predictions.show()

NameError: name 'predictions' is not defined

In [17]:
# Evaluation metrics
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="interaction_type", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

NameError: name 'predictions' is not defined

In [18]:
# Getting recommendations for users
user_recs = model.recommendForAllUsers(10)
user_recs.show()
user_recs = model.recommendForAllUsers(100).toPandas()

NameError: name 'model' is not defined

In [19]:
from py2neo import Graph, Node, Relationship

In [20]:
# Connecting to the Neo4j database
graph = Graph("bolt://localhost:7687", auth=("neo4j", "1234123412"))

ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://localhost:7687')

In [21]:
# Add Recommendations to Neo4j
for index, row in user_recs.iterrows():
    original_user_id = user_id_map[row['user_id']]
    user_node = graph.nodes.match("Customer", id=original_user_id).first()
    recommendations = row['recommendations']
    for rec in recommendations:
        original_product_id = product_id_map[rec['product_id']]
        score = rec['rating']
        product_node = graph.nodes.match("Product", id=original_product_id).first()
        if user_node and product_node:
            recommendation = Relationship(user_node, "RECOMMENDED", product_node, score=score)
            graph.create(recommendation)

NameError: name 'user_recs' is not defined